We want to categorize the countries in the 2019 happiness dataset. We will use the regions column in the 2015 dataset to achieve this.

In [0]:
input_path = "/FileStore/tables/2015.csv"
df = spark.read.format("csv").option("header", "true").load(input_path)


These are the top 20 happiest countries in 2015.

In [0]:
df.show()

+--------------------+--------------------+--------------+---------------+--------------+------------------------+-------+------------------------+-------+-----------------------------+----------+-----------------+
|             Country|              Region|Happiness Rank|Happiness Score|Standard Error|Economy (GDP per Capita)| Family|Health (Life Expectancy)|Freedom|Trust (Government Corruption)|Generosity|Dystopia Residual|
+--------------------+--------------------+--------------+---------------+--------------+------------------------+-------+------------------------+-------+-----------------------------+----------+-----------------+
|         Switzerland|      Western Europe|             1|          7.587|       0.03411|                 1.39651|1.34951|                 0.94143|0.66557|                      0.41978|   0.29678|          2.51738|
|             Iceland|      Western Europe|             2|          7.561|       0.04884|                 1.30232|1.40223|                 0

Categorize the countries into their respective regions.

In [0]:
df_py = df.collect()
reg_country = {}
countries = []
for i in df_py:
  if i["Region"] not in reg_country:
    reg_country[i["Region"]] = []
  reg_country[i["Region"]].append(i["Country"])
  countries.append(i["Country"])


for k in reg_country:
  print("\t", k)
  print(reg_country[k], "\n")

print("\tAll Countries\n", countries)


	 Western Europe
['Switzerland', 'Iceland', 'Denmark', 'Norway', 'Finland', 'Netherlands', 'Sweden', 'Austria', 'Luxembourg', 'Ireland', 'Belgium', 'United Kingdom', 'Germany', 'France', 'Spain', 'Malta', 'Italy', 'North Cyprus', 'Cyprus', 'Portugal', 'Greece'] 

	 North America
['Canada', 'United States'] 

	 Australia and New Zealand
['New Zealand', 'Australia'] 

	 Middle East and Northern Africa
['Israel', 'United Arab Emirates', 'Oman', 'Qatar', 'Saudi Arabia', 'Kuwait', 'Bahrain', 'Libya', 'Algeria', 'Turkey', 'Jordan', 'Morocco', 'Lebanon', 'Tunisia', 'Palestinian Territories', 'Iran', 'Iraq', 'Egypt', 'Yemen', 'Syria'] 

	 Latin America and Caribbean
['Costa Rica', 'Mexico', 'Brazil', 'Venezuela', 'Panama', 'Chile', 'Argentina', 'Uruguay', 'Colombia', 'Suriname', 'Trinidad and Tobago', 'El Salvador', 'Guatemala', 'Ecuador', 'Bolivia', 'Paraguay', 'Nicaragua', 'Peru', 'Jamaica', 'Dominican Republic', 'Honduras', 'Haiti'] 

	 Southeastern Asia
['Singapore', 'Thailand', 'Malaysia'

Now, we import the 2019 happiness dataset.

In [0]:
input_path2 = "/FileStore/tables/2019.csv"
df2 = spark.read.format("csv").option("header", "true").load(input_path2)

These are the 20 happiest countries in 2019.

In [0]:
df2.show()

+------------+-----------------+-----+--------------+--------------+-----------------------+----------------------------+----------+-------------------------+
|Overall rank|Country or region|Score|GDP per capita|Social support|Healthy life expectancy|Freedom to make life choices|Generosity|Perceptions of corruption|
+------------+-----------------+-----+--------------+--------------+-----------------------+----------------------------+----------+-------------------------+
|           1|          Finland|7.769|         1.340|         1.587|                  0.986|                       0.596|     0.153|                    0.393|
|           2|          Denmark|7.600|         1.383|         1.573|                  0.996|                       0.592|     0.252|                    0.410|
|           3|           Norway|7.554|         1.488|         1.582|                  1.028|                       0.603|     0.271|                    0.341|
|           4|          Iceland|7.494|        

Currently, all the data is stored as strings, which makes numerical calculations difficult. We will convert the columns into appropriate data types to make calculations easier to perform. We will also be removing any rows that have null values, to prevent any unexpected results.

In [0]:
from pyspark.sql.functions import col, to_timestamp

col_list = "Overall rank|Country or region|Score|GDP per capita|Social support|Healthy life expectancy|Freedom to make life choices|Generosity|Perceptions of corruption".split("|")

for i in col_list:
  if i == "Overall rank":
    df2 = df2.withColumn(i, col(i).cast("int"))
  elif i == "Country or region":
    df2 = df2.withColumn(i, col(i).cast("string"))
  else:
    df2 = df2.withColumn(i, col(i).cast("float"))

df_cleaned = df2.na.drop(subset=col_list)

We will make a list containing all the rows of the 2019 happiness dataset. This list will altered and converted back into a csv file.

These are the first 20 rows of this list.

In [0]:
data_list = []

df2_py = df_cleaned.collect()
for i in df2_py:
  data_list.append([])
  for c in i:
    data_list[-1].append(c)

print(data_list[:20])

[[1, 'Finland', 7.769000053405762, 1.340000033378601, 1.5870000123977661, 0.9860000014305115, 0.5960000157356262, 0.15299999713897705, 0.3930000066757202], [2, 'Denmark', 7.599999904632568, 1.3830000162124634, 1.5729999542236328, 0.9959999918937683, 0.5920000076293945, 0.25200000405311584, 0.4099999964237213], [3, 'Norway', 7.553999900817871, 1.4880000352859497, 1.5820000171661377, 1.027999997138977, 0.6029999852180481, 0.2709999978542328, 0.3409999907016754], [4, 'Iceland', 7.49399995803833, 1.3799999952316284, 1.6239999532699585, 1.0260000228881836, 0.5910000205039978, 0.3540000021457672, 0.11800000071525574], [5, 'Netherlands', 7.48799991607666, 1.3960000276565552, 1.5219999551773071, 0.9990000128746033, 0.5569999814033508, 0.32199999690055847, 0.2980000078678131], [6, 'Switzerland', 7.480000019073486, 1.4520000219345093, 1.5260000228881836, 1.0520000457763672, 0.5720000267028809, 0.2630000114440918, 0.34299999475479126], [7, 'Sweden', 7.3429999351501465, 1.3869999647140503, 1.48699

We can now convert the country names to regions. These are the countries and their respective region that will be replace the country name.

In [0]:
for i in data_list:
  for j in reg_country:
    if i[1] in reg_country[j]:
      i.insert(2, j)

ind = 0
while ind < 10:
  print(data_list[ind])
  ind+=1


[1, 'Finland', 'Western Europe', 7.769000053405762, 1.340000033378601, 1.5870000123977661, 0.9860000014305115, 0.5960000157356262, 0.15299999713897705, 0.3930000066757202]
[2, 'Denmark', 'Western Europe', 7.599999904632568, 1.3830000162124634, 1.5729999542236328, 0.9959999918937683, 0.5920000076293945, 0.25200000405311584, 0.4099999964237213]
[3, 'Norway', 'Western Europe', 7.553999900817871, 1.4880000352859497, 1.5820000171661377, 1.027999997138977, 0.6029999852180481, 0.2709999978542328, 0.3409999907016754]
[4, 'Iceland', 'Western Europe', 7.49399995803833, 1.3799999952316284, 1.6239999532699585, 1.0260000228881836, 0.5910000205039978, 0.3540000021457672, 0.11800000071525574]
[5, 'Netherlands', 'Western Europe', 7.48799991607666, 1.3960000276565552, 1.5219999551773071, 0.9990000128746033, 0.5569999814033508, 0.32199999690055847, 0.2980000078678131]
[6, 'Switzerland', 'Western Europe', 7.480000019073486, 1.4520000219345093, 1.5260000228881836, 1.0520000457763672, 0.5720000267028809, 0

The following countries will not be included in our final CSV file.

In [0]:
i = 0
while i < len(data_list):
  if len(data_list[i]) < 10:
    print(data_list.pop(i)[1])
  else:
    i += 1

Trinidad & Tobago
Northern Cyprus
North Macedonia
Somalia
Namibia
Gambia
South Sudan


This is the data

In [0]:
while i < len(data_list):
  print(data_list[i])
print(len(data_list))

149


Keep in mind that ranks such as 39, 64, and 84 no longer exist, as we have removed those countries from our dataset.

Now, we can convert these lists into a dataframe and export this as a CSV file. We will use this file in R to perform our data analysis.

In [0]:
col_names = "rank|country|region|score|gdp|support|life_expectancy|freedom|generosity|corruption".split("|")
df_happiness = spark.createDataFrame(data_list, col_names)
display(df_happiness)

rank,country,region,score,gdp,support,life_expectancy,freedom,generosity,corruption
1,Finland,Western Europe,7.769000053405762,1.340000033378601,1.5870000123977661,0.9860000014305115,0.5960000157356262,0.15299999713897705,0.3930000066757202
2,Denmark,Western Europe,7.599999904632568,1.3830000162124634,1.5729999542236328,0.9959999918937683,0.5920000076293945,0.25200000405311584,0.4099999964237213
3,Norway,Western Europe,7.553999900817871,1.4880000352859497,1.5820000171661377,1.027999997138977,0.6029999852180481,0.2709999978542328,0.3409999907016754
4,Iceland,Western Europe,7.49399995803833,1.3799999952316284,1.6239999532699585,1.0260000228881836,0.5910000205039978,0.3540000021457672,0.11800000071525574
5,Netherlands,Western Europe,7.48799991607666,1.3960000276565552,1.5219999551773071,0.9990000128746033,0.5569999814033508,0.32199999690055847,0.2980000078678131
6,Switzerland,Western Europe,7.480000019073486,1.4520000219345093,1.5260000228881836,1.0520000457763672,0.5720000267028809,0.2630000114440918,0.34299999475479126
7,Sweden,Western Europe,7.3429999351501465,1.3869999647140503,1.4869999885559082,1.0089999437332153,0.5740000009536743,0.2669999897480011,0.37299999594688416
8,New Zealand,Australia and New Zealand,7.307000160217285,1.3029999732971191,1.5570000410079956,1.0260000228881836,0.5849999785423279,0.33000001311302185,0.3799999952316284
9,Canada,North America,7.2779998779296875,1.3650000095367432,1.5049999952316284,1.0390000343322754,0.5839999914169312,0.2849999964237213,0.30799999833106995
10,Austria,Western Europe,7.245999813079834,1.3760000467300415,1.475000023841858,1.0160000324249268,0.5320000052452087,0.24400000274181366,0.22599999606609344
